 # 🎙️ 録音ファイル音声書き起こし

  ## 📝 使い方（3ステップで完了！）

  ### ステップ1️⃣：ランタイム設定
  1. 上部メニューの「ランタイム」をクリック
  2. 「ランタイムのタイプを変更」を選択
  3. 「ハードウェア アクセラレータ」を **「T4 GPU」** に変更
  4. 「保存」をクリック

  ### ステップ2️⃣：実行開始
  1. 上部メニューの「ランタイム」をクリック
  2. 「すべてのセルを実行」を選択
  3. セットアップが自動で開始されます（初回は3-5分かかります）

  ### ステップ3️⃣：ファイルアップロード
  1. 「ファイルを選択」ボタンが表示されたらクリック
  2. 音声ファイル（MP3/M4A/AAC）を選択
  3. 自動で書き起こしが開始されます
  4. 完了後、ダウンロードボタンをクリック

  ## 🔄 複数ファイルを処理する場合
  **2回目以降の実行前に必ず以下を実行してください：**
  1. 上部メニューの「ランタイム」をクリック
  2. 「ランタイムを再起動」を選択
  3. 再起動完了後、再度「すべてのセルを実行」

  *これを行わないとGPUメモリ不足エラーが発生します*

  ## ⚠️ 重要な注意点
  - **必ずT4 GPUを選択してください**（処理が大幅に高速化されます）
  - **Google Chromeブラウザを使用してください**（他のブラウザではファイルアップロードが不安
  定な場合があります）
  - 初回実行時は必要なソフトウェアのダウンロードで時間がかかりますが、2回目以降は高速です
  - 30分の音声ファイルの場合、10分前後書き起こし処理に時間がかかります

  ## 🎯 対応ファイル
  - **MP3**（.mp3）
  - **M4A**（.m4a）
  - **AAC**（.aac）

In [ ]:
# 🎙️ 音声書き起こしツール - 1セル完結版（高品質）

print("🎙️ 音声書き起こしツール起動中...\n")

# ライブラリインストール
print("📦 セットアップ中...")
!pip install -U openai-whisper ipywidgets -q
!sudo apt update -qq && sudo apt install ffmpeg -qq -y

import whisper
import subprocess
from google.colab import files
import time
import os
import ipywidgets as widgets
from IPython.display import display
import re

# 不要フレーズ除去関数
def clean_transcription(text):
    """不要なフレーズを除去し、自然な文章に整える"""

    # 除去するフレーズパターン
    unwanted_patterns = [
        r'^Thank you(?:\.)?$',
        r'^Thanks for listening(?:\.)?$',
        r'^That\'s all(?:\.)?$',
        r'^That\'s it(?:\.)?$',
        r'^Goodbye(?:\.)?$',
        r'^See you(?:\.)?$',
        r'^ご視聴ありがとうございました(?:。)?$',
        r'^ご清聴ありがとうございました(?:。)?$',
        r'^ありがとうございました(?:。)?$',
        r'^以上です(?:。)?$',
        r'^これで終わります(?:。)?$',
        r'^お疲れ様でした(?:。)?$',
    ]

    # 行ごとに分割して処理
    lines = text.split('\n')
    cleaned_lines = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # 不要フレーズのチェック
        is_unwanted = False
        for pattern in unwanted_patterns:
            if re.match(pattern, line, re.IGNORECASE):
                is_unwanted = True
                break

        if not is_unwanted:
            cleaned_lines.append(line)

    # テキストを結合
    cleaned_text = ' '.join(cleaned_lines)

    # 句読点の改善
    cleaned_text = improve_punctuation(cleaned_text)

    return cleaned_text

def improve_punctuation(text):
    """句読点を改善して自然な文章にする"""

    # 日本語の場合の処理
    if any(ord(char) > 127 for char in text):  # 日本語文字が含まれている場合
        # 長い文を適切に区切る
        text = re.sub(r'([。！？])\s*([あ-ん])', r'\1\n\2', text)
        text = re.sub(r'([です|ます|した|だ])\s+([あ-ん])', r'\1。\2', text)
        text = re.sub(r'([て|で|が|を|に|は])\s+', r'\1', text)
        # 改行を適切に処理
        text = re.sub(r'\n+', '\n', text)
    else:
        # 英語の場合の処理
        text = re.sub(r'([.!?])\s*([A-Z])', r'\1 \2', text)
        text = re.sub(r'([a-z])\s+([A-Z])', r'\1. \2', text)

    return text.strip()

# AIモデル準備
print("🤖 AIモデル準備中...")
#  | モデル      | 用途        | 処理時間 | 精度  |
#  |----------|-----------|------|-----|
#  | tiny     | 軽量・リアルタイム | 最速   | 低   |
#  | small    | 個人用・試験的   | 高速   | 中   |
#  | medium   | 一般的な用途    | 普通   | 高   |
#  | large-v3 | 本格的な書き起こし | やや遅い | 最高  |
#  | turbo    | 高速＋高精度    | 高速   | 高   |

model = whisper.load_model("large-v3")
print("✅ 準備完了!\n")

# ファイルアップロード
print("📂 音声ファイル（MP3/M4A/AAC）を選択してください:\n")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"\n✅ ファイル: {filename}")

    # 形式変換（必要時）
    audio_file = filename
    if filename.endswith(('.m4a', '.aac')):
        print("🔄 形式変換中...")
        audio_file = filename.rsplit('.', 1)[0] + '.mp3'
        subprocess.run(['ffmpeg', '-i', filename, audio_file, '-y'],
                       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 言語判定（ファイル名や内容から推測）
    is_japanese = any(ord(char) > 127 for char in filename) or 'jp' in filename.lower() or 'ja' in filename.lower()
    language = 'ja' if is_japanese else None

    # 書き起こし（改善されたパラメータ）
    print("\n🎯 書き起こし中...")
    start = time.time()

    # 言語に応じたプロンプト設定
    if language == 'ja':
        initial_prompt = "以下は、自然な会話形式の文章です。適切な句読点を含めて、読みやすく書き起こしてください。"
    else:
        initial_prompt = "The following is a natural conversation. Please transcribe with proper punctuation and readable format."

    result = model.transcribe(
        audio_file,
        language=language,
        word_timestamps=True,
        initial_prompt=initial_prompt,
        temperature=0.0,  # 一貫性を高める
        compression_ratio_threshold=2.4,  # 繰り返しを抑制
        logprob_threshold=-1.0,  # 品質の低いセグメントを除外
        no_speech_threshold=0.6  # 無音部分の処理改善
    )

    elapsed = time.time() - start
    print(f"✅ 完了！（{elapsed:.1f}秒）")

    # テキストの後処理
    print("🧹 テキスト品質改善中...")
    original_text = result["text"]
    cleaned_text = clean_transcription(original_text)

    # 結果ファイル作成
    output_file = filename.rsplit('.', 1)[0] + "_書き起こし.txt"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"ファイル: {filename}\n")
        f.write(f"処理時間: {elapsed:.1f}秒\n")
        f.write(f"言語: {'日本語' if language == 'ja' else '自動検出'}\n")
        f.write(f"文字数: {len(cleaned_text)}文字\n")
        f.write(f"生成日時: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("=" * 40 + "\n\n")
        f.write(cleaned_text)

    # 結果表示
    print(f"\n📝 結果（{len(cleaned_text)}文字）:")
    print("-" * 50)
    preview = cleaned_text[:300] if len(cleaned_text) > 300 else cleaned_text
    print(preview + ("..." if len(cleaned_text) > 300 else ""))
    print("-" * 50)

    # 改善点の表示
    removed_chars = len(original_text) - len(cleaned_text)
    if removed_chars > 0:
        print(f"🧹 品質改善: {removed_chars}文字の不要部分を除去")

    # ダウンロードボタン
    def download_result(b):
        files.download(output_file)
        b.description = '✅ ダウンロード開始'
        print(f"💾 ダウンロード: {output_file}")

    btn = widgets.Button(
        description='💾 結果をダウンロード',
        button_style='success',
        layout=widgets.Layout(width='200px', height='50px')
    )
    btn.on_click(download_result)

    print("\n👇 ダウンロードボタン 👇")
    display(btn)

    print(f"\n🎉 処理完了！ファイル: {output_file}")

    # 一時ファイル削除
    if audio_file != filename and os.path.exists(audio_file):
        os.remove(audio_file)

else:
    print("❌ ファイルがアップロードされませんでした")
    print("もう一度「すべてのセルを実行」してください")

🎙️ 音声書き起こしツール起動中...

📦 セットアップ中...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

100%|█████████████████████████████████████| 2.88G/2.88G [00:54<00:00, 56.5MiB/s]


✅ 準備完了!

📂 音声ファイル（MP3/M4A/AAC）を選択してください:



Saving 土曜ソマオ・ミートボール.aac to 土曜ソマオ・ミートボール.aac

✅ ファイル: 土曜ソマオ・ミートボール.aac
🔄 形式変換中...

🎯 書き起こし中...
✅ 完了！（38.2秒）
🧹 テキスト品質改善中...

📝 結果（542文字）:
--------------------------------------------------
この動画は、自然な会話形式の文章です。
ここまでだったということだ。
なぜなら、俺は大きな株を抜かせないための専門のマッチョ。驚いちゃっただろう。
それでは、説明しよう。俺は、この畑の持ち主に雇われた。
どっかにしよう。
どっかにしよう。
どっかにしよう。
おい!気になれ!おいおい、お前らこうなりすぎちゃってるぞ。
どっかにしよう。
どっかにしよう。
どっかにしよう。
どっかにしよう。
どっかにしよう。
どっかにしよう。
どっかにしよう。
どっかにしよう。
さすがに、関わる。
どっかにしよう。
こんなに気になられなかったのは、生まれて初めてだ。
そんなに黙らないと言うなら、こうするしかないよう...
--------------------------------------------------

👇 ダウンロードボタン 👇


Button(button_style='success', description='💾 結果をダウンロード', layout=Layout(height='50px', width='200px'), style=B…


🎉 処理完了！ファイル: 土曜ソマオ・ミートボール_書き起こし.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

💾 ダウンロード: 土曜ソマオ・ミートボール_書き起こし.txt
